In [1]:
from pulp import *
from pulp import LpProblem, LpVariable, LpMinimize, LpInteger, lpSum, value, LpBinary,LpStatusOptimal
import pulp
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore", message="Overwriting previously set objective.")
import utility
import docplex.mp.model
import docplex
import docplex_explainer
import mymetrics
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import joblib
import dill
from alibi.explainers import AnchorTabular


In [2]:
def load_data(dataset_name):
    if dataset_name == 'Iris':
        # Load Dataset
        dataset = datasets.load_iris()
        df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
        # Scale
        scaler = MinMaxScaler()
        scaler.fit(dataset.data)
        scaled_df = scaler.transform(dataset.data)
        # Check if binary targets
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
        targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Banknote':
        # Load Dataset
        df = pd.read_csv('./datasets/banknote_authentication.csv')
        # Scale
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Blood_Transfusion':
        df = pd.read_csv('./datasets/blood_transfusion.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Breast_Cancer':
        dataset = datasets.load_breast_cancer()
        df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
        scaler = MinMaxScaler()
        scaler.fit(dataset.data)
        scaled_df = scaler.transform(dataset.data)
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
        targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Climate':
        df = pd.read_csv('./datasets/climate_model_simulation_crashes.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Glass':
        df = pd.read_csv('./datasets/glass.csv')
        df['target'] = df['target'].apply(lambda x: 1 if x in [1, 2, 3] else 0)
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Ionosphere':
        df = pd.read_csv('./datasets/ionosphere.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Modeling':
        df = pd.read_csv('./datasets/User_Knowledge_Modeling.csv')
        df['target'] = df['target'].apply(lambda x: 1 if x == 'Low' else 0)
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)

    elif dataset_name == 'Parkinsons':
        df = pd.read_csv('./datasets/parkinsons.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)

    elif dataset_name == 'Pima':
        df = pd.read_csv('./datasets/diabetes.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)

    elif dataset_name == 'Sonar':
        df = pd.read_csv('./datasets/sonar.csv')
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
    elif dataset_name == 'Wine':
        dataset = datasets.load_wine()
        df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
        scaler = MinMaxScaler()
        scaler.fit(dataset.data)
        scaled_df = scaler.transform(dataset.data)
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
        targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)

    elif dataset_name == 'Vertebral-Column':
        dataset_name = 'Vertebral-Column'
        df = pd.read_csv('./datasets/column_2C.dat', sep=" ", names=['pelvic_incidence', 'pelvic_tilt', 'lumbar_lordosis_angle', 'sacral_slope', 'pelvic_radius', 'degree_spondylolisthesis','target'])
        df['target']=np.where(df['target']=='AB',1,0)
        scaler = MinMaxScaler()
        scaler.fit(df.values[:, :-1])
        scaled_df = scaler.transform(df.values[:, :-1])
        df_scaled = pd.DataFrame(scaled_df, columns=df.columns[:-1])
        targets = (utility.check_targets_0_1(df.values[:,-1])).astype(np.int32)
        df_scaled['target'] = targets
        X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
        return X_train, X_test, y_train, y_test, df.columns.values, np.unique(targets)
        
    else:
        print("Incorrect dataset name")

def parse_explanation(explanation, feature_names, epsilon=1e-6):
    bounds = [[0, 1] for _ in range(len(feature_names))]
    conditions = explanation

    for condition in conditions:
        condition_no_space = condition.replace(' ', '')  # for regex matching
        # Check for double inequality
        match = re.match(r'(\d+\.?\d*)\s*(<|<=)\s*([^\s<>]+)\s*(<|<=)\s*(\d+\.?\d*)', condition_no_space)
        
        if match:
            value_1, op1, feature_token, op2, value_2 = match.groups()
            value_1 = float(value_1)
            value_2 = float(value_2)
            lower_bound = value_1 if op1 == '<=' else value_1 + epsilon
            upper_bound = value_2 if op2 == '<=' else value_2
            upper_bound = upper_bound if op2 == '<=' else upper_bound - epsilon

            for idx, feature in enumerate(feature_names):
                if feature.replace(" ", "") in feature_token:
                    bounds[idx] = [lower_bound, upper_bound]
                    break
            continue  # go to next condition

        # Fallback to single operator logic
        for idx, feature in enumerate(feature_names):
            if feature in condition:
                cond_clean = condition.replace('<=', ' LESS_EQUAL ').replace('>=', ' GREATER_EQUAL ')
                cond_clean = cond_clean.replace('<', ' < ').replace('>', ' > ')
                tokens = cond_clean.split()

                tokens = ['<=' if token == 'LESS_EQUAL' else token for token in tokens]
                tokens = ['>=' if token == 'GREATER_EQUAL' else token for token in tokens]

                operator = None
                operator_pos = None
                for i, token in enumerate(tokens):
                    if token in ['>', '>=', '<', '<=']:
                        operator = token
                        operator_pos = i
                        break

                value = None
                if operator is not None and operator_pos is not None:
                    for i in range(operator_pos + 1, len(tokens)):
                        try:
                            value = float(tokens[i])
                            break
                        except ValueError:
                            continue

                if value is not None:
                    if operator == '>':
                        bounds[idx] = [value + epsilon, 1]
                    elif operator == '>=':
                        bounds[idx] = [value, 1]
                    elif operator == '<':
                        bounds[idx] = [0, value - epsilon]
                    elif operator == '<=':
                        bounds[idx] = [0, value]
                else:
                    print(f"Could not extract numeric value from condition: '{condition}'")

    return np.array(bounds)
def train_anchors(dataset_name):
    print(dataset_name)
    clf = joblib.load(f'models/{dataset_name}_svm_model.pkl')
    print(f'Loaded model')
    X_train, X_test, y_train, y_test,feature_names, class_names = load_data(dataset_name)
    if 'target' in feature_names:
        feature_names = feature_names[feature_names != 'target']
    print(f'feature names:\n {feature_names}')
    print(f'class names:\n {class_names}')
    predict_fn = lambda x: clf.predict(x)
    explainer = AnchorTabular(predict_fn, feature_names)
    explainer.fit(X_train, disc_perc=(25, 50, 75))
    return explainer, feature_names, class_names
    
# Compute means and standard deviations
def compute_mean_std(arr):
    return np.mean(arr), np.std(arr)

# Compute relative percentage differences
def relative_percentage_diff(new, old):
    if np.any(old == 0):
        print(f'Warning: found possible division by zero')
        return np.where(old != 0, ((new - old) / old) * 100, np.nan)
    return ((new - old) / old) * 100

In [3]:
datasets_name = ['Iris', 'Wine', 'Vertebral-Column', 'Pima', 'Parkinsons', 'Breast_Cancer', 'Blood_Transfusion', 'Ionosphere', 'Glass', 'Climate', 'Modeling', 'Banknote', 'Sonar']
p_values = [0.25,0.5,0.75]
for dataset_name in datasets_name:
    for p in p_values:
        path_anchors = f'./Anchors_results/{dataset_name}_results.csv' 
        path_twostep = dataset_name+ f'_results/results_{p}.csv'
        path_twostep_brute = dataset_name+ f'_results/raw_metric_data_{p}.csv'
        np.random.seed(50)
        anchors_brute_results = pd.read_csv(path_anchors)
        twostep_results = pd.read_csv(path_twostep)
        twostep_results_brute = pd.read_csv(path_twostep_brute)
        times_anchors = anchors_brute_results['time'].values
        coverage_anchors = anchors_brute_results['coverage'].values
        errors_anchors = anchors_brute_results['errors'].values
        rsum_anchors = anchors_brute_results['rsum'].values
        sizes_anchors = anchors_brute_results['size'].values
        
        times_twostep = twostep_results_brute['times_twostep'].values
        coverage_twostep = twostep_results_brute['coverage_twostep'].values
        rsum_twostep = twostep_results_brute['rsum_twostep'].values
        feature_sizes_twostep = twostep_results_brute['feature_sizes_twostep'].values
        
        
        
        
        
        # Ensure all lists are NumPy arrays
        times_twostep = np.array(times_twostep)
        coverage_twostep = np.array(coverage_twostep)
        sizes_anchors = np.array(sizes_anchors)
        feature_sizes_twostep = np.array(feature_sizes_twostep)
        rsum_anchors = np.array(rsum_anchors)
        rsum_twostep = np.array(rsum_twostep)
        
        
        
        # Compute means and standard deviations
        (time_mean_anchors, time_std_anchors) = compute_mean_std(times_anchors)
        (time_mean_twostep, time_std_twostep) = compute_mean_std(times_twostep)
        
        (coverage_mean_anchors, coverage_std_anchors) = compute_mean_std(coverage_anchors)
        (coverage_mean_twostep, coverage_std_twostep) = compute_mean_std(coverage_twostep)
        
        (sizes_mean_anchors, sizes_std_anchors) = compute_mean_std(sizes_anchors)
        (sizes_mean_twostep, sizes_std_twostep) = compute_mean_std(feature_sizes_twostep)
        
        (rsum_mean_anchors, rsum_std_anchors) = compute_mean_std(rsum_anchors)
        (rsum_mean_twostep, rsum_std_twostep) = compute_mean_std(rsum_twostep)
        
        # Compute relative percentage differences (Mean & Std)
        time_mean_diff = relative_percentage_diff(time_mean_twostep, time_mean_anchors)
        coverage_mean_diff = relative_percentage_diff(coverage_mean_twostep, coverage_mean_anchors)
        
        time_std_diff = relative_percentage_diff(time_std_twostep, time_std_anchors)
        coverage_std_diff = relative_percentage_diff(coverage_std_twostep, coverage_std_anchors)
        
        sizes_mean_diff = relative_percentage_diff(sizes_mean_twostep, sizes_mean_anchors)
        sizes_std_diff = relative_percentage_diff(sizes_std_twostep, sizes_std_anchors)
        
        rsum_mean_diff = relative_percentage_diff(rsum_mean_twostep, rsum_mean_anchors)
        rsum_std_diff = relative_percentage_diff(rsum_std_twostep, rsum_std_anchors)
        
        # Compute pointwise relative differences
        time_relative_pointwise = relative_percentage_diff(times_twostep, times_anchors)
        coverage_relative_pointwise = relative_percentage_diff(coverage_twostep, coverage_anchors)
        
        sizes_relative_pointwise = relative_percentage_diff(feature_sizes_twostep, sizes_anchors)
        rsum_relative_pointwise = relative_percentage_diff(rsum_twostep, rsum_anchors)
        
        # Compute pointwise means
        time_relative_mean = np.mean(time_relative_pointwise) 
        coverage_relative_mean = np.mean(coverage_relative_pointwise)
        sizes_relative_mean = np.mean(sizes_relative_pointwise)
        rsum_relative_mean = np.mean(rsum_relative_pointwise)
        
        # Compute pointwise standard deviations
        time_relative_std = np.std(time_relative_pointwise) 
        coverage_relative_std = np.std(coverage_relative_pointwise)
        sizes_relative_std = np.std(sizes_relative_pointwise)
        rsum_relative_std = np.std(rsum_relative_pointwise)
        
        # Organize Data
        all_metrics_data = all_metrics_data = {
            'Metric': ['Time', 'Size', 'Ranges_Sum', 'Coverage'],
            'ANCHORS_MEAN': [time_mean_anchors, sizes_mean_anchors, rsum_mean_anchors, coverage_mean_anchors],
            'ANCHORS_STD': [time_std_anchors, sizes_std_anchors, rsum_std_anchors, coverage_std_anchors],
            'TWOSTEP_MEAN': [time_mean_twostep, sizes_mean_twostep, rsum_mean_twostep, coverage_mean_twostep],
            'TWOSTEP_STD': [time_std_twostep, sizes_std_twostep, rsum_std_twostep, coverage_std_twostep],
            'MEAN_DIFF_%': [time_mean_diff, sizes_mean_diff, rsum_mean_diff, coverage_mean_diff],
            'STD_DIFF_%': [time_std_diff, sizes_std_diff, rsum_std_diff, coverage_std_diff],
            'POINTWISE_MEAN_%': [time_relative_mean, sizes_relative_mean, rsum_relative_mean, coverage_relative_mean],
            'POINTWISE_STD_%': [time_relative_std, sizes_relative_std, rsum_relative_std, coverage_relative_std],
            'ANCHORS_ERROR_RATE_MEAN%': [None, None, None, np.mean(errors_anchors/coverage_anchors)*100],
            'ANCHORS_ERROR_RATE_STD%': [None, None, None, np.std(errors_anchors/coverage_anchors)*100],
            
        }
        # Display and save
        all_metrics_df = pd.DataFrame(all_metrics_data)
        display(all_metrics_df)
        print(errors_anchors.sum())
        all_metrics_df.to_csv(f'Anchors_vs_Twostep_Results/Original_{dataset_name}_{p}_results.csv', index=False)

,Metric,ANCHORS_MEAN,ANCHORS_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%,ANCHORS_ERROR_RATE_MEAN%,ANCHORS_ERROR_RATE_STD%
0,Time,0.049050,0.036821,0.021891,0.004479,-55.370019,-87.836891,-17.938455,66.817053,NaN,NaN
1,Size,2.008850,0.644865,2.539823,0.498412,26.431718,-22.710708,42.109145,62.280497,NaN,NaN
2,Ranges_Sum,2.854955,0.439720,2.522860,0.444553,-11.632204,1.099110,-9.559153,21.211049,NaN,NaN
3,Coverage,26.318584,16.572322,35.044248,15.350991,33.154001,-7.369708,189.421580,382.768704,0.185442,0.804034


6


,Metric,ANCHORS_MEAN,ANCHORS_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%,ANCHORS_ERROR_RATE_MEAN%,ANCHORS_ERROR_RATE_STD%
0,Time,0.049050,0.036821,0.021932,0.003935,-55.286473,-89.313430,-18.268833,63.486520,NaN,NaN
1,Size,2.008850,0.644865,2.539823,0.498412,26.431718,-22.710708,42.109145,62.280497,NaN,NaN
2,Ranges_Sum,2.854955,0.439720,2.519221,0.446137,-11.759667,1.459260,-9.689744,21.245739,NaN,NaN
3,Coverage,26.318584,16.572322,35.557522,14.620230,35.104237,-11.779234,191.106726,379.732904,0.185442,0.804034


6


,Metric,ANCHORS_MEAN,ANCHORS_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%,ANCHORS_ERROR_RATE_MEAN%,ANCHORS_ERROR_RATE_STD%
0,Time,0.049050,0.036821,0.022266,0.004111,-54.606306,-88.835186,-18.388534,59.523360,NaN,NaN
1,Size,2.008850,0.644865,2.539823,0.498412,26.431718,-22.710708,42.109145,62.280497,NaN,NaN
2,Ranges_Sum,2.854955,0.439720,2.531183,0.439164,-11.340705,-0.126503,-9.259576,21.084520,NaN,NaN
3,Coverage,26.318584,16.572322,34.442478,15.870571,30.867518,-4.234481,185.748615,382.045038,0.185442,0.804034


...

46


,Metric,ANCHORS_MEAN,ANCHORS_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%,ANCHORS_ERROR_RATE_MEAN%,ANCHORS_ERROR_RATE_STD%
0,Time,4.206868,0.797166,0.300317,0.026482,-92.861281,-96.677985,-92.582874,1.659652,NaN,NaN
1,Size,2.775641,0.417159,51.647436,3.800817,1760.739030,811.118518,1811.965812,369.693195,NaN,NaN
2,Ranges_Sum,57.155959,1.145506,31.854303,2.597341,-44.267747,126.741774,-44.246174,4.671457,NaN,NaN
3,Coverage,3.294872,4.693446,1.000000,0.000000,-69.649805,-100.000000,-34.879380,37.976039,4.023083,12.221805


46


,Metric,ANCHORS_MEAN,ANCHORS_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%,ANCHORS_ERROR_RATE_MEAN%,ANCHORS_ERROR_RATE_STD%
0,Time,4.206868,0.797166,0.299299,0.029429,-92.885465,-96.308247,-92.609830,1.651189,NaN,NaN
1,Size,2.775641,0.417159,51.647436,3.800817,1760.739030,811.118518,1811.965812,369.693195,NaN,NaN
2,Ranges_Sum,57.155959,1.145506,31.852161,2.596723,-44.271496,126.687832,-44.249915,4.670499,NaN,NaN
3,Coverage,3.294872,4.693446,1.000000,0.000000,-69.649805,-100.000000,-34.879380,37.976039,4.023083,12.221805


46
